<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/rivaasrasr-finetuning-conformer-ctc-nemo/nvidia_logo.png" style="width: 90px; float: right;">

# How to Fine-Tune a Riva ASR Acoustic Model with NVIDIA NeMo
This tutorial walks you through how to fine-tune an NVIDIA Riva Parakeet acoustic model with NVIDIA NeMo.

## NVIDIA Riva Overview

NVIDIA Riva is a GPU-accelerated SDK for building speech AI applications that are customized for your use case and deliver real-time performance. <br/>
Riva offers a rich set of speech and natural language understanding (NLU) services such as:

- Automated speech recognition (ASR).
- Text-to-Speech synthesis (TTS).
- A collection of natural language processing (NLP) services, such as named entity recognition (NER), punctuation, and intent classification.

In this tutorial, we will fine-tune a Riva ASR acoustic model with NeMo. <br>
To understand the basics of Riva ASR APIs, refer to [Getting started with Riva ASR in Python](https://github.com/nvidia-riva/tutorials/blob/stable/asr-python-basics.ipynb). <br>

For more information about Riva, refer to the [Riva developer documentation](https://developer.nvidia.com/riva).

## NeMo (Neural Modules)
[NVIDIA NeMo](https://developer.nvidia.com/nvidia-nemo) is an open-source framework for building, training, and fine-tuning GPU-accelerated speech AI and NLU models with a simple Python interface. For information about how to set up NeMo, refer to the [NeMo GitHub](https://github.com/NVIDIA/NeMo) instructions.

In [ ]:
"""
You can run either this tutorial locally (if you have all the dependencies and a GPU) or on Google Colab.

Perform the following steps to setup in Google Colab:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub.
   a. Click **File** > **Upload Notebook** > **GITHUB** tab > copy/paste the GitHub URL.
3. Connect to an instance with a GPU.
   a. Click **Runtime** > Change the runtime type > select **GPU** for the hardware accelerator.
4. Run this cell to set up the dependencies.
5. Restart the runtime.
   a. Click **Runtime** > **Restart Runtime** for any upgraded packages to take effect.
"""

# Before running this notebook
   # pull docker container: docker pull nvcr.io/nvidia/cuda:12.4.1-cudnn-devel-ubuntu22.04 (recommended)
   # launch container and run below
      # `apt-get update`
      # install python3.10.12 and pip
         # `apt-get install -y --no-install-recommends   python3.10 python3.10-venv python3.10-distutils python3.10-dev`
         # `apt-get install -y --no-install-recommends python3-pip`
         # `update-alternatives --install /usr/bin/python python /usr/bin/python3.10 1`
         # `update-alternatives --set python /usr/bin/python3.10`
         # `python --version`

# run this notebook (you might need to run this cell two times where the site-packages need to be synced in first run)

# Install Dependencies
!pip3 install wget
!apt-get install -y wget sox libsndfile1 ffmpeg libsox-fmt-mp3 jq git # if failed, run `apt-get update` first
!pip3 install text-unidecode
!pip3 install matplotlib
!pip3 install Cython
!pip3 install torch==2.3.0
!pip3 install librosa


# Install NeMo
BRANCH = 'v2.0.0'
!python3 -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

# Modify some package for dependency consideration
!pip3 install numpy==1.24.1
!pip3 install --no-cache-dir huggingface-hub==0.24.5
!pip3 install transformers==4.40.2
!pip3 install numba==0.59.0
!pip3 install pytorch-lightning==2.3.3
!pip3 install lightning-utilities==0.10.1
!pip3 install omegaconf==2.3.0
!pip3 install jiwer==3.0.4

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, in the case where you want to use the "Run All Cells" (or similar) option,
uncomment `exit()` below to crash and restart the kernel.
"""
# exit()

---
## Fine-Tuning an ASR model with NeMo

### Download the Data

In this tutorial, we will use the popular AN4 dataset. Let's download it.

In [ ]:
! wget https://dldata-public.s3.us-east-2.amazonaws.com/an4_sphere.tar.gz  # for the original source, please visit http://www.speech.cs.cmu.edu/databases/an4/an4_sphere.tar.gz

After downloading, untar the dataset and move it to the correct directory.

In [ ]:
import os
DATA_DIR = os.getcwd()
os.environ["DATA_DIR"] = DATA_DIR
! tar -xvf an4_sphere.tar.gz
! mv an4 $DATA_DIR

### Pre-Processing

This step converts the `.mp3` files into `.wav` files and splits the data into training and testing sets. It also generates a "meta-data" file to be consumed by the data-loader for training and testing.

In [ ]:
import json, librosa, os, glob
import subprocess


source_data_dir = f"{DATA_DIR}/an4"
target_data_dir = f"{DATA_DIR}/an4_converted"

def an4_build_manifest(transcripts_path, manifest_path, target_wavs_dir):
    """Build an AN4 manifest from a given transcript file."""
    with open(transcripts_path, 'r') as fin:
        with open(manifest_path, 'w') as fout:
            for line in fin:
                # Lines look like this:
                # <s> transcript </s> (fileID)
                transcript = line[: line.find('(') - 1].lower()
                transcript = transcript.replace('<s>', '').replace('</s>', '')
                transcript = transcript.strip()

                file_id = line[line.find('(') + 1 : -2]  # e.g. "cen4-fash-b"
                audio_path = os.path.join(target_wavs_dir, file_id + '.wav')

                duration = librosa.core.get_duration(filename=audio_path)

                # Write the metadata to the manifest
                metadata = {"audio_filepath": audio_path, "duration": duration, "text": transcript}
                json.dump(metadata, fout)
                fout.write('\n')

"""Process AN4 dataset."""
if not os.path.exists(source_data_dir):
    link = 'http://www.speech.cs.cmu.edu/databases/an4/an4_sphere.tar.gz'
    raise ValueError(
        f"Data not found at `{source_data_dir}`. Please download the AN4 dataset from `{link}` "
        f"and extract it into the folder specified by the `source_data_dir` argument."
    )

# Convert SPH files to WAV files
sph_list = glob.glob(os.path.join(source_data_dir, '**/*.sph'), recursive=True)
target_wavs_dir = os.path.join(target_data_dir, 'wavs')
if not os.path.exists(target_wavs_dir):
    print(f"Creating directories for {target_wavs_dir}.")
    os.makedirs(os.path.join(target_data_dir, 'wavs'))

for sph_path in sph_list:
    wav_path = os.path.join(target_wavs_dir, os.path.splitext(os.path.basename(sph_path))[0] + '.wav')
    cmd = ["sox", sph_path, wav_path]
    subprocess.run(cmd, check=True)

# Build AN4 manifests
train_transcripts = os.path.join(source_data_dir, 'etc/an4_train.transcription')
train_manifest = os.path.join(target_data_dir, 'train_manifest.json')
an4_build_manifest(train_transcripts, train_manifest, target_wavs_dir)

test_transcripts = os.path.join(source_data_dir, 'etc/an4_test.transcription')
test_manifest = os.path.join(target_data_dir, 'test_manifest.json')
an4_build_manifest(test_transcripts, test_manifest, target_wavs_dir)


Let's listen to a sample audio file.

In [ ]:
# change path of the file here
import os
import IPython.display as ipd
path = os.environ["DATA_DIR"] + '/an4_converted/wavs/an268-mbmg-b.wav'
ipd.Audio(path)

### Tokenizers

If training dataset is not enough (<50 hrs) we recommend using tokenizers from pretrained model itself. The following tutorials explains how to use tokenizers from pretrained models for finetuning Parakeet models. If there's a change in vocab or you wish to train your own tokenizers you can use [NeMo tokenizer training script](https://github.com/NVIDIA/NeMo/blob/main/scripts/tokenizers/process_asr_text_tokenizer.py) and use [Hybrid model training script](https://github.com/NVIDIA/NeMo/blob/main/examples/asr/asr_hybrid_transducer_ctc/speech_to_text_hybrid_rnnt_ctc_bpe.py) to finetune the model on your data. Refer [asr-finetune-conformer-nemo](https://github.com/nvidia-riva/tutorials/blob/main/asr-finetune-conformer-ctc-nemo.ipynb) tutorial for more details.


#### Training Parakeet-Hybrid model

Hybrid RNNT-CTC models is a group of models with both the RNNT and CTC decoders. Training a hybrid model would speedup the convergence for the CTC models and would enable the user to use a single model which works as both a CTC and RNNT model. This category can be used with any of the ASR models. Hybrid models uses two decoders of CTC and RNNT on the top of the encoder.

NeMo uses `.yaml` files to configure the training parameters. You may update them directly by editing the configuration file or from the command-line interface. For example, if the number of epochs needs to be modified, along with a change in the learning rate, you can add `trainer.max_epochs=100` and `optim.lr=0.02` and train the model.

The following sample command uses the `speech_to_text_finetune.py` script in the `examples` folder to train/fine-tune a Parakeet-Hybrid ASR model for 1 epoch. For other ASR models like Citrinet, Conformer, you may find the appropriate config files in the NeMo GitHub repo under [examples/asr/conf/](https://github.com/NVIDIA/NeMo/tree/main/examples/asr/conf).


In [ ]:
# To fully train the model, you'll need to increase trainer.max_epochs from 1.
# Empirical evidence suggests that around 200 epochs should suffice.
NEMO_DIR = '/workspace/NeMo'
BRANCH='v2.0.0'
!git clone -b $BRANCH https://github.com/NVIDIA/NeMo $NEMO_DIR
!python $NEMO_DIR/examples/asr/speech_to_text_finetune.py \
        --config-path="../asr/conf/fastconformer/hybrid_transducer_ctc/" --config-name=fastconformer_hybrid_transducer_ctc_bpe \
        +init_from_pretrained_model=stt_en_fastconformer_hybrid_large_pc \
        ++model.train_ds.manifest_filepath="$DATA_DIR/an4_converted/train_manifest.json" \
        ++model.validation_ds.manifest_filepath="$DATA_DIR/an4_converted/test_manifest.json" \
        ++model.optim.sched.d_model=1024 \
        ++trainer.devices=1 \
        ++trainer.max_epochs=1 \
        ++trainer.precision=bf16 \
        ++model.optim.name="adamw" \
        ++model.optim.lr=0.1 \
        ++model.optim.weight_decay=0.001 \
        ++model.optim.sched.warmup_steps=100 \
        ++exp_manager.version=test \
        ++exp_manager.use_datetime_version=False \
        ++exp_manager.exp_dir=$DATA_DIR/checkpoints

In [9]:
nemo_file_path = os.path.join(DATA_DIR, 'checkpoints/FastConformer-Hybrid-Transducer-CTC-BPE/test/checkpoints/FastConformer-Hybrid-Transducer-CTC-BPE.nemo')

### ASR Evaluation

Now that we have a model trained, we need to check how well it performs.

In [ ]:
!python $NEMO_DIR/examples/asr/speech_to_text_eval.py \
    model_path=$nemo_file_path \
    dataset_manifest=$DATA_DIR/an4_converted/test_manifest.json \
    output_filename=./test_manifest_predictions.json \
    batch_size=32 \
    amp=True


### ASR Model Export

With NeMo, you can also export your model in a format that can be deployed using NVIDIA Riva: a highly performant application framework for multi-modal conversational AI services using GPUs. The same command for exporting to ONNX can be used here. The only small variation is the configuration for `export_format` in the spec file.
The model above is trained with hybrid loss which means, it is a fusion of both CTC and RNNT model. We need to extract a specific decoder to use in RIVA. We can use [convert_nemo_asr_hybrid_to_ctc.py](https://github.com/NVIDIA/NeMo/blob/main/examples/asr/asr_hybrid_transducer_ctc/helpers/convert_nemo_asr_hybrid_to_ctc.py) script to extract the required decoder.


In [ ]:
ctc_model_path = os.path.join(DATA_DIR, 'checkpoints/FastConformer-Hybrid-Transducer-CTC-BPE/test/checkpoints/FastConformer-CTC-BPE_ctc.nemo')
! python3 $NEMO_DIR/examples/asr/asr_hybrid_transducer_ctc/helpers/convert_nemo_asr_hybrid_to_ctc.py -i $nemo_file_path -o $ctc_model_path -t ctc

#### Install the Packages

We will now install the NeMo and `nemo2riva` packages. `nemo2riva` is available on [NVIDIA NGC](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/riva/resources/riva_quickstart/files?version=2.8.1). Make sure you install NGC CLI first before running the following commands.

In [ ]:
!pip install nvidia-pyindex
!ngc registry resource download-version "nvidia/riva/riva_quickstart:"$__riva_version__
!pip install nemo2riva==2.19.0
!pip install protobuf==3.20.0

#### Convert to Riva

Convert the downloaded model to the `.riva` format. We will set the encryption key with `--key=nemotoriva`. Choose a different encryption key value when generating `.riva` models for production.

For deploying the RNNT model in RIVA, use `nemo2riva --out $riva_file_path --key=nemotoriva --format nemo $rnnt_file_path`

In [ ]:
riva_file_path = ctc_model_path[:-5]+".riva"
!nemo2riva --key=nemotoriva --max-dim 1000 --onnx-opset 18 --out $riva_file_path $ctc_model_path

## More Resources
You can find more information about working with NeMo's ASR models in the [ASR section](https://github.com/NVIDIA/NeMo/tree/main/tutorials/asr) of the NeMo tutorials.

## What's Next?

You can use NeMo to build custom models for your own applications, and deploy them with NVIDIA Riva! Refer to the [Conformer-CTC deployment tutorial](https://github.com/nvidia-riva/tutorials/blob/main/asr-deployment-conformer-ctc.ipynb).